In [1]:
import pandas as pd
import numpy as np
import env
import matplotlib.pyplot as plt
import seaborn as sns 
import os
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import wrangle

In [98]:
def new_zillow_data(SQL_query, url):
    '''
    this function will:
    - take in a SQL_query 
    -create a connection url to mySQL
    -return a df of the given query from the zillow database
    
    '''
    
    url= f'mysql+pymysql://{env.username}:{env.password}@{env.hostname}/zillow'
    return pd.read_sql(SQL_query,url)    
    
    
    
def get_zillow_data(filename = "zillow_data.csv"):
    '''
    this function will:
    -check local directory for csv file
        return if exists
    if csv doesn't exist
    if csv doesnt exist:
        - create a df of the SQL_query
        write df to csv
    output zillow df
    
    '''
    directory = os.getcwd()
    
    SQL_query = '''select bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, 
                    yearbuilt, taxamount, fips, propertylandusedesc from properties_2017
                    join propertylandusetype using(propertylandusetypeid)
                    where propertylandusedesc like 'Single Family Residential';'''
    
    filename = "zillow_data.csv"
    
    url= f'mysql+pymysql://{env.username}:{env.password}@{env.hostname}/zillow'

    if os.path.exists(directory + filename):
        df = pd.read_csv(filename)
        return df
    else:
        df= new_zillow_data(SQL_query, url)
        df.to_csv(filename)
        return df
    
    
def preparing_data_zillow(df):
    '''
    droping unwanted rows for zillow first exercise
    converting float columns to integers
    '''
    df = df.dropna()
    df.calculatedfinishedsquarefeet = df.calculatedfinishedsquarefeet.astype(int)
    df.fips = df.fips.astype(int)
    df.yearbuilt = df.yearbuilt.astype(int)
    return df


def wrangle_zillow():
    '''
    gets and prepares zillow data
    '''
    df = get_zillow_data()
    df = preparing_data_zillow(df)
    return df



def split_data(df, target):
    '''
    Takes in a dataframe and returns train, validate, test subset dataframes
    '''
    
    
    train, test = train_test_split(df,
                                   test_size=.2, 
                                   random_state=123, 
                                   stratify=df[target]
                                   )
    train, validate = train_test_split(train, 
                                       test_size=.25, 
                                       random_state=123, 
                                       stratify=train[target]
                                       )
    
    return train, validate, test

In [3]:
df = wrangle.wrangle_zillow()


In [8]:
df.calculatedfinishedsquarefeet = df.calculatedfinishedsquarefeet.astype(int)
df.fips = df.fips.astype(int)
df.yearbuilt = df.yearbuilt.astype(int)

df

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,propertylandusedesc
4,4.0,2.0,3633,296425.0,2005,6941.39,6037,Single Family Residential
6,3.0,4.0,1620,847770.0,2011,10244.94,6037,Single Family Residential
7,3.0,2.0,2077,646760.0,1926,7924.68,6037,Single Family Residential
11,0.0,0.0,1200,5328.0,1972,91.60,6037,Single Family Residential
14,0.0,0.0,171,6920.0,1973,255.17,6037,Single Family Residential
...,...,...,...,...,...,...,...,...
2152856,4.0,4.0,4375,422400.0,2015,13877.56,6037,Single Family Residential
2152858,4.0,3.0,2262,960756.0,2015,13494.52,6059,Single Family Residential
2152859,4.0,4.5,3127,536061.0,2014,6244.16,6059,Single Family Residential
2152861,3.0,2.5,1974,424353.0,2015,5302.70,6059,Single Family Residential


In [82]:
# df = df.drop(columns = 'propertylandusedesc')

# taxvaluedollarcnt is structurevaluedollarcnt + landtaxvaluedollarcnt
# calculatedfinishedsquarefeet seems to get its value from 1 of three finished square feet columns


def preparing_data(df):
    '''
    droping unwanted rows
    '''
    df = df.dropna()
    return df

In [89]:
df = remove_values_no_data(df)


df.isnull().sum() 

[ True  True  True ...  True  True  True]


/var/folders/9q/9mllqnsn2vq07btq58zw9zzc0000gn/T/ipykernel_77522/3788022007.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  the_df = the_df[df.bathroomcnt == 0.0]
/var/folders/9q/9mllqnsn2vq07btq58zw9zzc0000gn/T/ipykernel_77522/3788022007.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  the_df = the_df[df.calculatedfinishedsquarefeet.isnull()]
/var/folders/9q/9mllqnsn2vq07btq58zw9zzc0000gn/T/ipykernel_77522/3788022007.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  the_df = the_df[df.taxvaluedollarcnt.isnull()]
/var/folders/9q/9mllqnsn2vq07btq58zw9zzc0000gn/T/ipykernel_77522/3788022007.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  the_df = the_df[df.yearbuilt.isnull()]
/var/folders/9q/9mllqnsn2vq07btq58zw9zzc0000gn/T/ipykernel_77522/3788022007.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  the_df = the

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8121
taxvaluedollarcnt                130
yearbuilt                       8974
taxamount                       4079
fips                               0
dtype: int64

In [94]:
the_df = df.dropna()
the_df

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0
...,...,...,...,...,...,...,...
2152856,4.0,4.0,4375.0,422400.0,2015.0,13877.56,6037.0
2152858,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059.0
2152859,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059.0
2152861,3.0,2.5,1974.0,424353.0,2015.0,5302.70,6059.0
